In [ ]:
# %load main.py
import argparse
from midi2audio import FluidSynth
import tensorflow as tf
import os
import numpy as np
from model import GenreLSTM
import mido
import midi_util
from mido import MidiFile
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
def setup_dir():
    

    print('[*] Setting up directory...')

    main_path = "./runs"
    current_run = os.path.join(main_path, "test")

    files_path = "./data"
    files_path = os.path.join(files_path, "test")

    x_path = os.path.join(files_path, 'inputs')
    y_path = os.path.join(files_path, 'velocities')
    eval_path = os.path.join(files_path, 'eval')

    model_path = os.path.join(current_run, 'model')
    logs_path = os.path.join(current_run, 'tmp')
    png_path = os.path.join(current_run, 'png')
    pred_path = os.path.join(current_run, 'predictions')

    if not os.path.exists(current_run):
        os.makedirs(current_run)
    if not os.path.exists(logs_path):
        os.makedirs(logs_path)
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    if not os.path.exists(png_path):
        os.makedirs(png_path)
    if not os.path.exists(pred_path):
        os.makedirs(pred_path)

    dirs = {
            'main_path': main_path,
            'current_run': current_run,
            'model_path': model_path,
            'logs_path': logs_path,
            'png_path': png_path,
            'eval_path': eval_path,
            'pred_path': pred_path,
            'x_path': x_path,
            'y_path': y_path
        }

    # print main_path
    # print current_run
    # print model_path
    # print logs_path
    # print png_path
    # print eval_path
    # print x_path
    # print y_path
    return dirs


fs = FluidSynth('~/sf.sf2')



In [2]:
def plot_evaluation(epoch, filename, c_out, j_out, e_out, out_list, path=True):
    fig = plt.figure(figsize=(14,11), dpi=120)
    fig.suptitle(filename, fontsize=10, fontweight='bold')

    graph_items = [out_list[-1]*120, c_out[-1]*120, j_out[-1]*120,  (c_out[-1]-j_out[-1])*120 , e_out[-1]]
    plots = len(graph_items)
    cmap = ['jet', 'jet', 'jet', 'jet', 'bwr']
    vmin = [0,0,0,-10,-1]
    vmax = [120,120,120,10,1]
    names = ["Actual", "Classical", "Jazz", "Difference", "Encoded"]


    for i in xrange(0, plots):
        fig.add_subplot(1,plots,i+1)
        plt.imshow(graph_items[i], vmin=vmin[i], vmax=vmax[i], cmap=cmap[i], aspect='auto')

        a = plt.colorbar(aspect=80)
        a.ax.tick_params(labelsize=7)
        ax = plt.gca()
        ax.xaxis.tick_top()

        if i == 0:
            ax.set_ylabel('Time Step')
        ax.xaxis.set_label_position('top')
        ax.tick_params(axis='both', labelsize=7)
        fig.subplots_adjust(top=0.85)
        ax.set_title(names[i], y=1.09)
        # plt.tight_layout()

    plt.xlim(0,88)

    if not path:
        out_png = os.path.join(self.dirs['png_path'], filename.split('.')[0] + "-e%d" % (epoch)+".png")
        plt.savefig(out_png, bbox_inches='tight')
        plt.close(fig)
    else:
#         out_png = os.path.join(path) + "predict.png")
#         plt.savefig(path, bbox_inches='tight')
        plt.show()

In [3]:
dirs = setup_dir()
network  = GenreLSTM(dirs, input_size=176, mini=True, bi=True)
network.prepare_model()




[*] Setting up directory...
[*] Preparing bidirectional dynamic RNN...
INFO:tensorflow:Summary name Jazz error is illegal; using Jazz_error instead.
INFO:tensorflow:Summary name Classical error is illegal; using Classical_error instead.
INFO:tensorflow:Summary name Average error is illegal; using Average_error instead.
INFO:tensorflow:Summary name Jazz negation is illegal; using Jazz_negation instead.
INFO:tensorflow:Summary name Classical negation is illegal; using Classical_negation instead.


In [4]:
network.load("a", path="/Users/Iman/Desktop/models/model/model-e212.ckpt")

 [*] Loading checkpoint...


In [5]:
import file_util

In [6]:
file_util.validate_data("/Users/Iman/Desktop/models/mid/",4)
file_util.quantize_data("/Users/Iman/Desktop/models/mid_valid/",4)
file_util.save_data("/Users/Iman/Desktop/models/mid_valid_quantized/", 4, one_hot=True)


Processed 0 files out of 0
Processed 0 files out of 0

Processed 0 files out of 0


In [7]:
input_dir = "/Users/Iman/Desktop/models/inputs/"
output_dir = "/Users/Iman/Desktop/models/velocities/"
jazz_names = []
classical_names = []

In [8]:
for i, filename in enumerate(os.listdir(os.path.join(input_dir, "jazz"))):
    if filename.split(".")[-1] == "npy":
        jazz_names.append(filename)

        
for i, filename in enumerate(os.listdir(os.path.join(input_dir, "classical"))):
    if filename.split(".")[-1] == "npy":
        classical_names.append(filename)
# c_error, c_out, j_out, e_out, out_list = network.predict("/Users/Iman/Desktop/models/mid_valid_quantized_inputs/n2.mid.npy", "/Users/Iman/Desktop/models/mid_valid_quantized_velocities/n2.mid.npy")

In [9]:
print classical_names
print jazz_names

['beethoven_les_adieux_2_format0.mid.npy', 'burg_gewitter_format0.mid.npy', 'chpn-p23_format0.mid.npy', 'chpn_op25_e1_format0.mid.npy', 'debussy_cc_5_format0.mid.npy', 'franceschetti02.mid.npy', 'K570 Piano Sonata 2mov.mid.npy', 'LEE_J02.mid.npy', 'pavane.mid.npy', 'rac_op3_2_format0.mid.npy', 'reflect.mid.npy', 'schu_143_1_format0.mid.npy', 'TET01.mid.npy', 'UENO01.mid.npy', 'victoria03.mid.npy', 'waldstein_1_format0.mid.npy', 'YOO01.mid.npy', 'ZHOU01.mid.npy']
['alfiepno.mid.npy', 'Autumn In NY.mid.npy', 'Effendi - McCoy Tyner.mid.npy', 'Falling Grace .mid.npy', 'INVITATION.mid.npy', 'My Romance.mid.npy', 'NoMoreBlues.mid.npy', 'Remember - solo jazz piano.mid.npy', 'Renewaltrio.mid.npy', 'SammyWalked.mid.npy', 'Speak Low (Doug McKenzie).mid.npy', 'taketheatrain.mid.npy', 'Very Early solo piano.mid.npy', 'Warmvalleysolo.mid.npy', 'WhyDoILoveYou.mid.npy', 'xmas.mid.npy', 'YBSN2CH2.mid.npy', 'Zingaro - Jobim.mid.npy']


In [10]:
for filename in classical_names:
    print filename
    name = filename.split(".")[0] + ".mid"
    classical_input = os.path.join("/Users/Iman/Desktop/models/inputs/classical", filename)
    classical_velocities = os.path.join("/Users/Iman/Desktop/models/velocities/classical", filename)
    c_error, c_out, j_out, e_out, out_list = network.predict(classical_input, classical_velocities)
#     plot_evaluation(1, "n2", c_out, j_out, e_out, out_list)
    mid = MidiFile(os.path.join("/Users/Iman/Desktop/models/predict/original/classical/", name))
    classical = midi_util.stylify(mid, c_out[-1], 4)
    classical.save(os.path.join("/Users/Iman/Desktop/models/predict/original_classical/classical/", name))
    
    mid = MidiFile(os.path.join("/Users/Iman/Desktop/models/predict/original/classical/", name))
    jazz = midi_util.stylify(mid, j_out[-1], 4)
    jazz.save(os.path.join("/Users/Iman/Desktop/models/predict/original_classical/jazz/", name))

beethoven_les_adieux_2_format0.mid.npy
burg_gewitter_format0.mid.npy
chpn-p23_format0.mid.npy
chpn_op25_e1_format0.mid.npy
debussy_cc_5_format0.mid.npy
franceschetti02.mid.npy
K570 Piano Sonata 2mov.mid.npy
LEE_J02.mid.npy
pavane.mid.npy
rac_op3_2_format0.mid.npy
reflect.mid.npy
schu_143_1_format0.mid.npy
TET01.mid.npy
UENO01.mid.npy
victoria03.mid.npy
waldstein_1_format0.mid.npy
YOO01.mid.npy
ZHOU01.mid.npy


In [11]:
for filename in jazz_names:
    print filename
    name = filename.split(".")[0] + ".mid"
    jazz_input = os.path.join("/Users/Iman/Desktop/models/inputs/jazz", filename)
    jazz_velocities = os.path.join("/Users/Iman/Desktop/models/velocities/jazz", filename)
    c_error, c_out, j_out, e_out, out_list = network.predict(jazz_input, jazz_velocities)
    mid = MidiFile(os.path.join("/Users/Iman/Desktop/models/predict/original/jazz/", name))
    classical = midi_util.stylify(mid, c_out[-1], 4)
#     plot_evaluation(0, "filename", c_out, j_out, e_out, out_list)
    classical.save(os.path.join("/Users/Iman/Desktop/models/predict/original_jazz/classical/", name))
    mid = MidiFile(os.path.join("/Users/Iman/Desktop/models/predict/original/jazz/", name))
    jazz = midi_util.stylify(mid, j_out[-1], 4)
    jazz.save(os.path.join("/Users/Iman/Desktop/models/predict/original_jazz/jazz/", name))



alfiepno.mid.npy
Autumn In NY.mid.npy
Effendi - McCoy Tyner.mid.npy
Falling Grace .mid.npy
INVITATION.mid.npy
My Romance.mid.npy
NoMoreBlues.mid.npy
Remember - solo jazz piano.mid.npy
Renewaltrio.mid.npy
SammyWalked.mid.npy
Speak Low (Doug McKenzie).mid.npy
taketheatrain.mid.npy
Very Early solo piano.mid.npy
Warmvalleysolo.mid.npy
WhyDoILoveYou.mid.npy
xmas.mid.npy
YBSN2CH2.mid.npy
Zingaro - Jobim.mid.npy


In [12]:
# np.save("/Users/Iman/Desktop/models/jazz_out.npy", j_out)

In [13]:
# mid = MidiFile("/Users/Iman/Desktop/models/mid_valid_quantized/n2.mid")
# mid = midi_util.quantize(mid,4)

In [14]:
# a = midi_util.stylify(mid, c_out[-1], 4)

In [15]:
# a.save("/Users/Iman/Desktop/models/class_mid.mid")

In [16]:
# fs.midi_to_audio('input.mid', 'output.wav')


In [17]:
for filename in classical_names:
#     fs.midi_to_audio('input.mid', 'output.wav')
    print filename
    name = filename.split(".")[0] + ".mid"
    name_m = filename.split(".")[0] + ".wav"

    
    mid_classical = "/Users/Iman/Desktop/models/predict/original_classical/classical/"
    mid_jazz = "/Users/Iman/Desktop/models/predict/original_classical/jazz/"

    output_classical = "/Users/Iman/Desktop/models/predict/original_classical/classical/wav"
    output_jazz = "/Users/Iman/Desktop/models/predict/original_classical/jazz/wav"
    
    c_mp3 = os.path.join(output_classical, name_m)
    j_mp3 = os.path.join(output_jazz, name_m)
    
    c_mid = os.path.join(mid_classical, name)
    j_mid = os.path.join(mid_jazz, name)
    
    fs.midi_to_audio(c_mid, c_mp3)
    fs.midi_to_audio(j_mid, j_mp3)


    


    




beethoven_les_adieux_2_format0.mid.npy
burg_gewitter_format0.mid.npy
chpn-p23_format0.mid.npy
chpn_op25_e1_format0.mid.npy
debussy_cc_5_format0.mid.npy
franceschetti02.mid.npy
K570 Piano Sonata 2mov.mid.npy
LEE_J02.mid.npy
pavane.mid.npy
rac_op3_2_format0.mid.npy
reflect.mid.npy
schu_143_1_format0.mid.npy
TET01.mid.npy
UENO01.mid.npy
victoria03.mid.npy
waldstein_1_format0.mid.npy
YOO01.mid.npy
ZHOU01.mid.npy


In [18]:
for filename in jazz_names:
    name = filename.split(".")[0] + ".mid"
    name_m = filename.split(".")[0] + ".wav"

    output_classical = "/Users/Iman/Desktop/models/predict/original_jazz/classical/wav"
    output_jazz = "/Users/Iman/Desktop/models/predict/original_jazz/jazz/wav"

    mid_classical = "/Users/Iman/Desktop/models/predict/original_jazz/classical/"
    mid_jazz = "/Users/Iman/Desktop/models/predict/original_jazz/jazz/"

    c_mp3 = os.path.join(output_classical, name_m)
    j_mp3 = os.path.join(output_jazz, name_m)
    
    c_mid = os.path.join(mid_classical, name)
    j_mid = os.path.join(mid_jazz, name)
    
    fs.midi_to_audio(c_mid, c_mp3)
    fs.midi_to_audio(j_mid, j_mp3)





In [19]:
jazz_input = "/Users/Iman/Desktop/fri/input.npy"
jazz_velocities = "/Users/Iman/Desktop/fri/output.npy"



# name = filename.split(".")[0] + ".mid"
# jazz_input = os.path.join("/Users/Iman/Desktop/models/inputs/jazz", filename)
# jazz_velocities = os.path.join("/Users/Iman/Desktop/models/velocities/jazz", filename)
c_error, c_out, j_out, e_out, out_list = network.predict(jazz_input, jazz_velocities)
mid = MidiFile("/Users/Iman/Desktop/fri/friday.mid")
classical = midi_util.stylify(mid, j_out[-1], 4)
classical.save("/Users/Iman/Desktop/fri/fri_out_j.mid")


In [33]:
jazz_input = "/Users/Iman/Desktop/input.npy"
jazz_velocities = "/Users/Iman/Desktop/output.npy"



# name = filename.split(".")[0] + ".mid"
# jazz_input = os.path.join("/Users/Iman/Desktop/models/inputs/jazz", filename)
# jazz_velocities = os.path.join("/Users/Iman/Desktop/models/velocities/jazz", filename)
c_error, c_out, j_out, e_out, out_list = network.predict(jazz_input, jazz_velocities)
mid = MidiFile("/Users/Iman/Desktop/yi.mid")
classical = midi_util.stylify(mid, c_out[-1], 4)
classical.save("/Users/Iman/Desktop/yiruma_c.mid")


In [36]:

fs.midi_to_audio("/Users/Iman/Desktop/yi.mid", "/Users/Iman/Desktop/yi.wav")
fs.midi_to_audio("/Users/Iman/Desktop/yiruma_c.mid", "/Users/Iman/Desktop/yiruma_c.wav")

myaudio = AudioSegment.from_wav("/Users/Iman/Desktop/yi.wav") 
myaudio = match_target_amplitude(myaudio, -20.0)
myaudio.export("/Users/Iman/Desktop/yi.wav", format="wav")

myaudio = AudioSegment.from_wav("/Users/Iman/Desktop/yiruma_c.wav") 
myaudio = match_target_amplitude(myaudio, -20.0)
myaudio.export("/Users/Iman/Desktop/yiruma_c.wav", format="wav")




<open file '/Users/Iman/Desktop/yiruma_c.wav', mode 'wb+' at 0x116e9cc90>

In [20]:
# pydub.AudioSegment.ffmpeg = "/Users/Iman/ffmpeg"

c_error, c_out, j_out, e_out, out_list = network.predict(jazz_input, jazz_velocities)
mid = MidiFile("/Users/Iman/Desktop/fri/friday.mid")
classical = midi_util.stylify(mid, c_out[-1], 4)
classical.save("/Users/Iman/Desktop/fri/fri_out_c.mid")

In [28]:
import pydub
from pydub import AudioSegment
from pydub.utils import make_chunks

    
j_mid_classical = "/Users/Iman/Desktop/models/predict/original_jazz/classical/wav"
j_mid_jazz = "/Users/Iman/Desktop/models/predict/original_jazz/jazz/wav"
c_mid_jazz = "/Users/Iman/Desktop/models/predict/original_classical/jazz/wav"
c_mid_classical = "/Users/Iman/Desktop/models/predict/original_classical/classical/wav"
o_mid_jazz = "/Users/Iman/Desktop/models/predict/original/jazz/wav"
o_mid_classical = "/Users/Iman/Desktop/models/predict/original/classical/wav"

songs = "/Users/Iman/Desktop/models/predict/songs/"

firs = [j_mid_classical, j_mid_jazz, c_mid_jazz, c_mid_classical]
# firs = [o_mid_jazz, o_mid_classical]

In [29]:
# firs = [o_mid_jazz, o_mid_classical]
dict = {
j_mid_classical : "oc",
j_mid_jazz : "oj",
c_mid_jazz : "oj",
c_mid_classical : "oc",
o_mid_jazz : "aj",
o_mid_classical : "ac"
}

def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    return sound.apply_gain(change_in_dBFS)



In [30]:
directory = o_mid_classical
for folder in firs:
    for filename in os.listdir(folder):
        if filename.split('.')[-1] == "wav": 
            name = filename.split('.')[0]
            path = os.path.join(folder, filename)
            print path
            myaudio = AudioSegment.from_wav(path) 
            chunk_length_ms = 10000 # pydub calculates in millisec
            chunks = make_chunks(myaudio, chunk_length_ms) #Make chunks of one sec

        #Export all of the individual chunks as wav files

            song_path = os.path.join(songs, name)
            if not os.path.exists(song_path):
                os.makedirs(song_path)
            
            
            for i, chunk in enumerate(chunks):
                normalized_chunk = match_target_amplitude(chunk, -20.0)
                chunk_name = os.path.join(song_path, name + "_{0}".format(i) + "_" + dict[folder] + ".wav")
                print "exporting", chunk_name
                normalized_chunk.export(chunk_name, format="wav")

/Users/Iman/Desktop/models/predict/original_jazz/classical/wav/alfiepno.wav
exporting /Users/Iman/Desktop/models/predict/songs/alfiepno/alfiepno_0_oc.wav
exporting /Users/Iman/Desktop/models/predict/songs/alfiepno/alfiepno_1_oc.wav
exporting /Users/Iman/Desktop/models/predict/songs/alfiepno/alfiepno_2_oc.wav
exporting /Users/Iman/Desktop/models/predict/songs/alfiepno/alfiepno_3_oc.wav
exporting /Users/Iman/Desktop/models/predict/songs/alfiepno/alfiepno_4_oc.wav
exporting /Users/Iman/Desktop/models/predict/songs/alfiepno/alfiepno_5_oc.wav
exporting /Users/Iman/Desktop/models/predict/songs/alfiepno/alfiepno_6_oc.wav
exporting /Users/Iman/Desktop/models/predict/songs/alfiepno/alfiepno_7_oc.wav
exporting /Users/Iman/Desktop/models/predict/songs/alfiepno/alfiepno_8_oc.wav
exporting /Users/Iman/Desktop/models/predict/songs/alfiepno/alfiepno_9_oc.wav
exporting /Users/Iman/Desktop/models/predict/songs/alfiepno/alfiepno_10_oc.wav
exporting /Users/Iman/Desktop/models/predict/songs/alfiepno/alfie

In [24]:
# myaudio = AudioSegment.from_wav("/Users/Iman/Desktop/models/predict/original/classical/waldstein_1_format0.mid.wav")
# chunk_length_ms = 10000 # pydub calculates in millisec
# chunks = make_chunks(myaudio, chunk_length_ms) #Make chunks of one sec

# #Export all of the individual chunks as wav files

# for i, chunk in enumerate(chunks):
#     chunk_name = "{0}".format(i) + ".mid.wav"
#     print "exporting", chunk_name
# #         chunk.export(chunk_name, format="wav")